# 04. Semantic Kernel

In this lab, we will introduce **Semantic Kernel**. Like **Langchain**, it provides a framework for working with AI models and supports applications written in Python. Semantic Kernel also adds support for .NET and Java applications.

As with the other labs, we'll start by reading in values from the `.env` file.

___
**NOTE**: This lab uses **.NET** so be sure to select .NET and not Python when starting to use this notebook.
___

In [1]:
#r "nuget: dotenv.net, 3.2.0"

using dotenv.net;

// Read values from .env file
var envVars = DotEnv.Fluent()
    .WithoutExceptions()
    .WithEnvFiles("../../../.env")
    .WithTrimValues()
    .WithDefaultEncoding()
    .WithOverwriteExistingVars()
    .WithoutProbeForEnv()
    .Read();

// Load values into variables and strip quotes
var model = envVars["AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"].Replace("\"", "");
var azureEndpoint = envVars["AZURE_OPENAI_ENDPOINT"].Replace("\"", "");
var apiKey = envVars["AZURE_OPENAI_API_KEY"].Replace("\"", "");


Installed Packages dotenv.net, 3.2.0

To start working with Semantic Kernel, we'll need to load its nuget package.

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.19.0"

Installed Packages Microsoft.SemanticKernel, 1.19.0

Semantic Kernel works by creating an instance of the Kernel and then adding in various plugins to perform different functions. Those plugins or functions can then be called individually or chained together to perform more complex tasks.

We use the standard .NET `builder` pattern to initialise the kernel. Notice that we pass in the details of the completion model that we're going to use, the Azure OpenAI API endpoint URL and the API key.

In [3]:
using Microsoft.SemanticKernel;

var builder = Kernel.CreateBuilder();
builder.Services.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
var kernel = builder.Build();

## Send a prompt to Azure OpenAI using Semantic Kernel

Now that we've established a connection to the Azure OpenAI API, we can go ahead and send a prompt to the LLM.

In [4]:
Console.WriteLine(await kernel.InvokePromptAsync("What things could I make with a Raspberry Pi?"));

Raspberry Pi is an extremely versatile single-board computer that can be used for a wide range of projects. Here are some popular and interesting projects you can undertake with a Raspberry Pi:

1. **Media Center**: Turn your Raspberry Pi into a media center using software like Kodi, LibreELEC, or OSMC to stream videos, music, and more.

2. **Retro Gaming Console**: Use software such as RetroPie to transform your Raspberry Pi into a retro gaming console capable of playing classic games from systems like NES, SNES, and Sega Genesis.

3. **Home Automation System**: Implement smart home automation using tools like Home Assistant or OpenHAB to control smart devices like lights, thermostats, and security systems.

4. **Network Attached Storage (NAS)**: Set up a personal file storage solution by connecting an external hard drive to your Raspberry Pi and using software like OpenMediaVault.

5. **Web Server**: Host your own website or web applications by setting up a LAMP (Linux, Apache, MySQL

Let's take that simple prompt forward and create a function with a prompt template to perform a simple request to Azure OpenAI. The template allows us to define a prompt and add placeholders for values that we will provide later. These values could come from user input, or another function, for example.

In [5]:
var whatCanIMakeFunction = kernel.CreateFunctionFromPrompt(
    new PromptTemplateConfig()
    {
        Template = @"What interesting things can I make with a {{$item}}?",
        InputVariables = [
            new() { Name = "item", Description = "An item to make something with.", IsRequired=true }
        ]
    });


`{{$item}}` represents the value we will provide later and will be replaced when we make the call to Azure OpenAI.

Next, we'll define a value for `item` and then call the function, passing in the `item` as we do so.

In [6]:
string item = "raspberry pi";

var response = await kernel.InvokeAsync(whatCanIMakeFunction, new () { { "item", item }});

Console.WriteLine(response);

A Raspberry Pi is a versatile and affordable platform that can be used to create a wide range of interesting projects. Here are some ideas to inspire you:

1. **Retro Gaming Console**: Use software like RetroPie to turn your Raspberry Pi into a retro gaming console. Play classic games from systems like NES, SNES, Sega Genesis, and more.

2. **Home Automation System**: Build a smart home system with home automation services like Home Assistant or OpenHAB. Control lights, thermostats, security cameras, and other IoT devices.

3. **Media Center**: Set up a media center using Kodi or Plex. Stream movies, TV shows, and music to your TV or play content stored on your local network.

4. **Network-Attached Storage (NAS)**: Create your own NAS system to store and share files across your home network using software like OpenMediaVault.

5. **Ad-Blocking Server**: Install Pi-hole to block ads on all devices connected to your network. It acts as a DNS sinkhole, preventing unwanted ads from reachin

The result is a list of ideas for things we could make with a Raspberry Pi. 

We could then use Semantic Kernel to take that list of ideas and summarize it. First, we'll create a new prompt that will generate a summary of some text.

In [7]:
var thingsToMakeSummary = kernel.CreateFunctionFromPrompt(
    new PromptTemplateConfig()
    {
        Template = @"Summarize the following text: {{$thingsToMake}}?",
        InputVariables = [
            new() { Name = "thingsToMake", Description = "A list of things you could make.", IsRequired=true }
        ]
    });

Next, we can call the kernel again, this time passing in the list of ideas, the `response` from the previous call, as the value for `thingsToMake`.

In [8]:
var summary = await kernel.InvokeAsync(thingsToMakeSummary, new () { { "thingsToMake", response }});

Console.WriteLine(summary);

A Raspberry Pi is a versatile and cost-effective platform suitable for a variety of creative projects. Here are some ideas:

1. **Retro Gaming Console**: Convert it into a console for classic games using RetroPie.
2. **Home Automation System**: Use Home Assistant or OpenHAB for smart home control.
3. **Media Center**: Set up streaming with Kodi or Plex.
4. **Network-Attached Storage (NAS)**: Share files with OpenMediaVault.
5. **Ad-Blocking Server**: Block ads network-wide using Pi-hole.
6. **Security Camera System**: Use MotionEyeOS for home surveillance.
7. **Personal Web Server**: Host websites with Apache or Nginx.
8. **Weather Station**: Collect local weather data with sensors.
9. **Voice Assistant**: Create a voice assistant using Mycroft, Google Assistant, or Alexa.
10. **Robotics**: Build and program robots.
11. **DIY Arcade Machine**: Combine with an arcade cabinet for gaming.
12. **Network Monitor**: Monitor traffic with Nagios.
13. **Smart Mirror**: Create a mirror displayin

## Summary

Semantic Kernel is another example of an AI orchestrator. Like other orchestrators, it can be used to simplify the process of creating complex AI applications.

## Next Section

📣 [AI Orchestration](../../03-orchestration/README.md)